# TensorFlow Recommenders: Quickstart

In this tutorial, we build a simple matrix factorization model using the [MovieLens 100K dataset](https://grouplens.org/datasets/movielens/100k/) with TFRS. We can use this model to recommend movies for a given user.

# Import TensorFlow Recommender System (TFRS)

In [ ]:
!pip install -q sagemaker==2.9.2
!pip install -q sagemaker-experiments==0.1.24
!pip install -q tensorflow==2.3.0
!pip install -q tensorflow-recommenders==0.2.0
!pip install -q tensorflow-datasets==4.0.0

In [ ]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [ ]:
class RankingModel(tf.keras.Model):

  def __init__(self, embedding_dimension, unique_user_ids, unique_movie_titles):
    super().__init__()

    # Compute embeddings for users.
    self.user_model = tf.keras.Sequential([
      tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # Compute embeddings for movies.
    self.movie_model = tf.keras.Sequential([
      tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])

    # Compute predictions.
    self.ratings = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
  ])
    
  def call(self, inputs):

    user_id, movie_title = inputs

    user_model = self.user_model(user_id)
    movie_model = self.movie_model(movie_title)

    return self.ratings(tf.concat([user_model, movie_model], axis=1))


In [ ]:
class MovielensModel(tfrs.models.Model):

  def __init__(self, embedding_dimension, unique_user_ids, unique_movie_titles):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel(embedding_dimension, unique_user_ids, unique_movie_titles)
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    rating_predictions = self.ranking_model(
        (features["user_id"], features["movie_title"]))

    # The task computes the loss and the metrics.
    return self.task(labels=features["user_rating"], predictions=rating_predictions)

# Load Ratings

In [ ]:
ratings = tfds.load('movielens/100k-ratings',                     
                    download=False,
                    data_dir='./tensorflow_datasets/',
                    split='train')
print(ratings)

In [ ]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"]
})
print(ratings)

# Load Movies

In [ ]:
movies = tfds.load('movielens/100k-movies', 
                   download=False,                   
                   data_dir='./tensorflow_datasets/',
                   split='train')
print('Movies BEFORE', movies)

In [ ]:
movies = movies.map(lambda x: x["movie_title"])
print('Movies AFTER', movies)

In [ ]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [ ]:
movie_titles = ratings.batch(100_000).map(lambda x: x["movie_title"])
user_ids = ratings.batch(100_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [ ]:
embedding_dimension = 32

In [ ]:
# task = tfrs.tasks.Ranking(
#   loss = tf.keras.losses.MeanSquaredError(),
#   metrics=[tf.keras.metrics.RootMeanSquaredError()]
# )

In [ ]:
model = MovielensModel(embedding_dimension, unique_user_ids, unique_movie_titles)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [ ]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [ ]:
epochs = 1

In [ ]:
model.fit(cached_train, epochs=epochs)

In [ ]:
model.evaluate(cached_test, return_dict=True)

# Make Predictions
Use brute-force search to set up retrieval using the trained representations.

In [ ]:
index = tfrs.layers.factorized_top_k.BruteForce(model.ranking_model.user_model)
index.index(movies.batch(100).map(model.ranking_model.movie_model), movies)

In [ ]:
import os

local_model_dir_bruteforce_model = './exported_models/bruteforce_model/'

tensorflow_saved_model_path_bruteforce_model = os.path.join(local_model_dir_bruteforce_model, 'tensorflow/saved_model/0')

os.makedirs(tensorflow_saved_model_path_bruteforce_model, exist_ok=True)

In [ ]:
k = 5
user_id = "42"

_, titles = index(np.array([user_id]))

print(f"Top {k} recommendations for user {user_id}: {titles[0, :k]}")

In [ ]:
print('Compiled model {}'.format(index))          
print(index.summary())

In [ ]:
index.save(tensorflow_saved_model_path_bruteforce_model, save_format='tf')

In [ ]:
!saved_model_cli show --all --dir $tensorflow_saved_model_path_bruteforce_model

In [ ]:
user_id = "42"

!saved_model_cli run --input_exprs 'input_1=np.array(["$user_id"])' --tag_set serve --signature_def serving_default --dir ./model/tensorflow/saved_model/0